## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

Requirement already up-to-date: pythainlp in ./opt/anaconda3/lib/python3.8/site-packages (2.3.1)


In [3]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [4]:
df = pd.read_csv('Wongnai Reviews - Small.csv')

In [9]:
df.head()

,Review ID,Review,Review_tokenized
0,1,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...,"คน,ชอบ,ทาน,Macchiato,เป็นประจำ,นึง,เดิน,เข้าไป..."
1,2,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...,"Art,of,Coffee,Kasetsart,ร้านกาแฟ,รสชาติ,เยี่ยม..."
2,3,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...,"กวง,ทะเล,เผา,อาหารทะเล,เค้า,สด,เนื้อ,ปู,หวาน,ค..."
3,4,วันนี้มีโอกาสตื่นเช้าครับเลยถึงโอกาสออกมาหาอะไ...,"มีโอกาส,ตื่น,เช้า,โอกาส,มาหา,อร่อย,ทาน,ตอนเช้า..."
4,5,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...,"ชอบ,ทาน,กิน,อาหาร,เวียดนาม,บ้าน,ซอย,ราชครู,เข้..."


## **Tokenize Words**

In [10]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [11]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [12]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


## **Create Dictionary**

In [13]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [14]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', '-', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', '

In [15]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [16]:
num_topics = 7
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 6.33 s, sys: 50.1 ms, total: 6.38 s
Wall time: 6.57 s


In [18]:
gensimvis.prepare(model, gensim_corpus, dictionary, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6     -29.594002  60.112713       1        1  29.870812
3     -45.996586  11.891685       2        1  28.670433
4      20.368856  70.019325       3        1  13.348057
0      53.928768  31.704309       4        1   9.747517
5     -12.437127 -26.423267       5        1   9.522111
1       3.965971  21.797720       6        1   5.025061
2      37.524284 -16.517805       7        1   3.816010, topic_info=            Term        Freq       Total Category  logprob  loglift
914       ไข่มุก   98.000000   98.000000  Default  30.0000  30.0000
2           กาแฟ  276.000000  276.000000  Default  29.0000  29.0000
36            ชา  237.000000  237.000000  Default  28.0000  28.0000
68            กก   76.000000   76.000000  Default  27.0000  27.0000
438           นม  138.000000  138.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
171         เค้า    4.342820  117.651163   Topic7  -5.7347  -0.0332
826  เครื่องดื่ม    4.208045  107.467217   Topic7  -5.7663   0.0258
17          สั่ง    4.334860  265.433888   Topic7  -5.7366  -0.8487
265         บ้าน    3.735525   45.007945   Topic7  -5.8854   0.7770
538       สำหรับ    3.732314   95.658731   Topic7  -5.8862   0.0222

[545 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
3691      5  0.837139                                                   
318       1  0.454679                                                  "
318       2  0.277859                                                  "
318       3  0.227339                                                  "
318       4  0.025260                                                  "
...     ...       ...                                                ...
649       2  0.883414                                               ไลน์
5400      2  0.150796                                                  –
5400      3  0.603186                                                  –
5400      6  0.150796                                                  –
3148      7  0.496760  …................................................

[1046 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 5, 1, 6, 2, 3])

In [19]:
model.show_topic(3)

[('อร่อย', 0.017467633),
 ('ทาน', 0.015669363),
 ('รสชาติ', 0.013050843),
 ('ดี', 0.01212705),
 ('อาหาร', 0.009963087),
 ('เมนู', 0.00783945),
 ('สั่ง', 0.007822054),
 ('-', 0.0073312866),
 ('กิน', 0.0071977307),
 ('ราคา', 0.006678272)]

In [20]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [22]:
df[['topics','score']]

,topics,score
0,3,0.993763
1,5,0.994876
2,3,0.188361
3,6,0.997329
4,2,0.996410
...,...,...
295,0,0.998859
296,5,0.996890
297,6,0.996476
298,6,0.998807
